In [1]:
import pandas as pd
from datetime import datetime, timezone, timedelta
import csv
import os
import requests
import json
import time

In [2]:
# kr_high = pd.read_csv('data/01_puuids/kr_puuids_high_tiers.csv')

In [3]:
# os.mkdir('data/02_match-v5')

In [2]:
'''패치 일자'''
kst = timezone(timedelta(hours=9))
startTime_25_1 = datetime(2025, 1, 9, 7, 0, 0, tzinfo=kst)
startTime_25_5 = datetime(2025, 3, 5, 3, 30, 0, tzinfo=kst)
startTime_25_9 = datetime(2025, 4, 30, 7, 30, 0, tzinfo=kst)
startTime_25_13 = datetime(2025, 6, 25, 7, 0, 0, tzinfo=kst)
startTime_25_17 = datetime(2025, 8, 27, 8, 30, 0, tzinfo=kst)
startTime_25_21 = datetime(2025, 10, 22, 7, 40, 0, tzinfo=kst)
startTime_26_1 = datetime(2026, 1, 8, 7, 20, 0, tzinfo=kst)
startTime_26_2 = datetime(2026, 2, 4, 6, 0, 0, tzinfo=kst)
# print(int(startTime.timestamp()))

# 액트 기준으로 구분하기
patch_times = {
    "period_1": (startTime_25_1, startTime_25_5), # 25_1 ~ 25_5 전
    "period_2": (startTime_25_5, startTime_25_9), # 25_5 ~ 25_9 전
    "period_3": (startTime_25_9, startTime_25_13), # 25_9 ~ 26_13 전
    "period_4": (startTime_25_13, startTime_25_17), # 25_13 ~ 25_17 전
    "period_5": (startTime_25_17, startTime_25_21), # 25_17 ~ 25_21 전
    "period_6": (startTime_25_21, startTime_26_1), # 25_21 ~ 26_1 전
    "period_7": (startTime_26_1, startTime_26_2) # 26_1 ~ 26_2
}

In [3]:
# periods = []
# for name, (start, end) in patch_times.items():
#     periods.append({
#         "name": name,
#         "start_ts": int(start.timestamp()),
#         "end_ts": int(end.timestamp()) if end else None
#     })

In [4]:
def unix_conv(unix_time):
    dt = datetime.datetime.utcfromtimestamp(unix_time)
    dt = dt.strftime('%Y-%m-%d %H:%M:%S')
    return dt

- 함수 모은거

In [5]:
API_KEY=""
HEADERS = {"X-Riot-Token": API_KEY}

REGION_MAPPING = {
    'KR': 'asia', 'JP1': 'asia',
    'BR1': 'americas', 'NA1': 'americas', 'LA1': 'americas', 'LA2': 'americas',
    'EUN1': 'europe', 'EUW1': 'europe', 'TR1': 'europe', 'RU': 'europe', 'ME1': 'europe',
    'OC1': 'sea', 'SG2': 'sea', 'TW2': 'sea', 'VN2': 'sea'
}

OUTPUT_FILE = "data/02_match-v5/collected_match_ids.csv"
PROGRESS_FILE = "data/02_match-v5/processed_puuids.txt"  # 수집 완료된 유저 저장용
SAVE_INTERVAL = 10

# def get_matches_for_one_user(puuid, platform, startTime, total_count=100):
#     routing = REGION_MAPPING.get(platform.upper(), 'asia')
#     url = f"https://{routing}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids"
    
#     user_match_ids = []
#     for start_index in range(0, total_count, 100):
#         params = {
#             "api_key" : API_KEY,
#             # "queue": 420,
#             "startTime": startTime,
#             # "endTime": endTime,
#             "start": start_index, 
#             "count": 100}
#         try:
#             response = requests.get(url, headers=HEADERS, params=params)
#             if response.status_code == 200:
#                 fetched_ids = response.json()
#                 user_match_ids.extend(fetched_ids)
#                 if len(fetched_ids) < 100: break # 더 이상 데이터 없음
#             elif response.status_code == 429:
#                 print("!! Rate Limit! 10초 대기...")
#                 time.sleep(10)
#                 continue # 현재 구간 다시 시도
#             else:
#                 break
#             time.sleep(0.05) # Personal Key 속도 준수
#         except Exception as e:
#             print(f"오류: {e}")
#             break
#     return user_match_ids

# def save_to_csv(match_id_set, file_name):
#     """
#     수집된 매치 ID 세트를 CSV에 저장 (중복 제거 포함)
#     """
#     new_df = pd.DataFrame(list(match_id_set), columns=['match_id'])
    
#     # 파일이 없으면 새로 만들고(header 포함), 있으면 이어 쓰기(header 제외)
#     if not os.path.exists(file_name):
#         new_df.to_csv(file_name, index=False, mode='w', encoding='utf-8')
#     else:
#         new_df.to_csv(file_name, index=False, mode='a', header=False, encoding='utf-8')
    
#     print(f"--- [중간 저장 완료] {len(new_df)}개의 새로운 매치 ID 저장됨 ---")

# def collect_with_checkpoints(df_dict):
#     current_session_matches = set() # 현재 세션에서의 중복 방지용
    
#     for platform, df in df_dict.items():
#         print(f"\n🚀 {platform} 지역 수집 시작...")
        
#         for idx, puuid in enumerate(df['puuid']):
#             m_ids = get_matches_for_one_user(puuid, platform)
#             current_session_matches.update(m_ids)
            
#             # 중간 저장 로직: SAVE_INTERVAL(10명) 마다 실행
#             if (idx + 1) % SAVE_INTERVAL == 0:
#                 print(f"[{idx+1}/{len(df)}] 진행 중...", end=' ')
#                 save_to_csv(current_session_matches, OUTPUT_FILE)
#                 # 저장 후 세션을 비워주면 메모리 부담이 줄어듭니다.
#                 current_session_matches.clear() 

#         # 지역 하나가 끝나면 남은 데이터 저장
#         if current_session_matches:
#             save_to_csv(current_session_matches, OUTPUT_FILE)
#             current_session_matches.clear()

#     # 마지막으로 전체 파일에서 혹시 모를 중복 제거
#     final_cleanup(OUTPUT_FILE)

# def final_cleanup(file_name):
#     print("\n🧹 전체 데이터 중복 제거 작업 시작...")
#     df = pd.read_csv(file_name)
#     before_count = len(df)
#     df.drop_duplicates(subset=['match_id'], inplace=True)
#     df.to_csv(file_name, index=False)
#     print(f"✅ 최종 완료! ({before_count} -> {len(df)}개)")

# # 실행
# # collect_with_checkpoints(df_dict)

# def load_processed_puuids():
#     """이미 수집 완료된 유저 목록을 불러옵니다."""
#     if os.path.exists(PROGRESS_FILE):
#         with open(PROGRESS_FILE, 'r') as f:
#             # 한 줄씩 읽어서 세트에 저장 (중복 제거 및 빠른 검색용)
#             return set(line.strip() for line in f)
#     return set()

# def save_progress(puuid_list):
#     """수집 완료된 유저 목록을 파일에 추가합니다."""
#     # 경로가 없으면 생성
#     os.makedirs(os.path.dirname(PROGRESS_FILE), exist_ok=True)
#     with open(PROGRESS_FILE, 'a') as f:
#         for puuid in puuid_list:
#             f.write(f"{puuid}\n")

# def collect_with_resume(df_dict, startTime):
#     # 1. 작업 완료 명단 로드
#     processed_puuids = load_processed_puuids()
#     print(f"📊 기존 작업 기록 확인: {len(processed_puuids)}명의 유저는 이미 수집되었습니다.")

#     current_session_matches = set()
#     newly_processed_this_session = [] # 이번 세션에서 새로 완료한 유저들

#     for platform, df in df_dict.items():
#         print(f"\n🚀 {platform} 지역 수집 시작...")
        
#         for idx, puuid in enumerate(df['puuid']):
#             # 2. 건너뛰기 로직
#             if puuid in processed_puuids:
#                 continue 

#             # 3. 데이터 수집
#             m_ids = get_matches_for_one_user(puuid, platform, startTime, total_count=100)
#             current_session_matches.update(m_ids)
#             newly_processed_this_session.append(puuid)
            
#             # 4. 중간 저장 (SAVE_INTERVAL 마다)
#             if len(newly_processed_this_session) >= SAVE_INTERVAL:
#                 print(f"[{idx+1}/{len(df)}] 저장 중...", end=' ')
#                 save_to_csv(current_session_matches, OUTPUT_FILE)
#                 save_progress(newly_processed_this_session) # 유저 명단도 저장
                
#                 # 메모리 비우기
#                 current_session_matches.clear()
#                 processed_puuids.update(newly_processed_this_session)
#                 newly_processed_this_session = []

#         # 지역 종료 후 남은 데이터 처리
#         if newly_processed_this_session:
#             save_to_csv(current_session_matches, OUTPUT_FILE)
#             save_progress(newly_processed_this_session)
#             current_session_matches.clear()
#             newly_processed_this_session = []

#     print("\n✨ 모든 지역 수집 프로세스가 종료되었습니다.")

# ==============================================================================================================
# 260209 수정
# 16주 간격 수집
# 각 기간 유저마다 100개 matchid 
# ==============================================================================================================
def get_matches_by_period(puuid, platform, start_dt, end_dt):
    """
    start_dt, end_dt: datetime 객체 (함수 내부에서 timestamp로 변환)
    """
    routing = REGION_MAPPING.get(platform.upper(), 'asia') # 기본값 asia 설정
    url = f"https://{routing}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids"
    
    # datetime 객체를 timestamp(int)로 변환
    start_ts = int(start_dt.timestamp())
    end_ts = int(end_dt.timestamp())

    params = {
        # "api_key": API_KEY, # 헤더에 넣었으므로 파라미터에서는 제외해도 됨
        "type": 'ranked',
        "startTime": start_ts,
        "endTime": end_ts,
        "count": 100
    }
    
    try:
        # Rate Limit 방지 딜레이 (필요시 조절)
        time.sleep(1.2) 
        
        response = requests.get(url, headers=HEADERS, params=params)
        
        if response.status_code == 200:
            return response.json()
            
        elif response.status_code == 429:
            # [수정됨] 헤더에서 Retry-After 값을 가져와 대기
            retry_after = int(response.headers.get("Retry-After", 10)) # 없으면 기본 10초
            print(f"⚠️ Rate Limit Exceeded (429). Sleeping for {retry_after} seconds...")
            time.sleep(retry_after + 1) # 안전을 위해 1초 더 대기
            return get_matches_by_period(puuid, platform, start_dt, end_dt) # 재시도
            
        else:
            print(f"Error {response.status_code}: {response.text}")
            return []
            
    except Exception as e:
        print(f"Request Exception: {e}")
        return []

# 수정된 저장 함수 (period 정보 포함)
def save_to_csv_with_period(match_data_list, file_name):
    # 경로가 없으면 생성
    os.makedirs(os.path.dirname(file_name), exist_ok=True)
    
    new_df = pd.DataFrame(match_data_list, columns=['match_id', 'period'])
    if not os.path.exists(file_name):
        new_df.to_csv(file_name, index=False, mode='w', encoding='utf-8')
    else:
        new_df.to_csv(file_name, index=False, mode='a', header=False, encoding='utf-8')


def final_cleanup_by_period(file_name):
    if not os.path.exists(file_name):
        return
    df = pd.read_csv(file_name)
    initial_len = len(df)
    df.drop_duplicates(subset=['match_id'], inplace=True)
    df.to_csv(file_name, index=False)
    print(f"✅ 최종 정리 완료. {initial_len} -> {len(df)} (고유 매치 수)")

# --- 메인 실행 로직 ---

def collect_by_patch_periods(df_dict):
    processed_puuids = load_processed_puuids()
    current_session_data = [] 
    newly_processed_puuids = []

    for platform, df in df_dict.items():
        print(f"\n🚀 {platform} 지역 수집 시작... (총 {len(df)}명)")
        
        for idx, puuid in enumerate(df['puuid']):
            if puuid in processed_puuids:
                continue

            # [수정됨] patch_times 딕셔너리 순회 방식 수정 (periods 변수 오류 해결)
            for period_name, (p_start, p_end) in patch_times.items():
                m_ids = get_matches_by_period(puuid, platform, p_start, p_end)
                
                for m_id in m_ids:
                    current_session_data.append((m_id, period_name))
                
            newly_processed_puuids.append(puuid)
            
            # 중간 저장 로직
            if len(newly_processed_puuids) >= SAVE_INTERVAL:
                save_to_csv_with_period(current_session_data, OUTPUT_FILE)
                save_progress(newly_processed_puuids)
                
                print(f"[{idx+1}/{len(df)}] 진행 중.. 저장 완료 (이번 턴 매치: {len(current_session_data)}개)")
                
                current_session_data = []
                processed_puuids.update(newly_processed_puuids)
                newly_processed_puuids = []

    # 남은 데이터 저장
    if newly_processed_puuids or current_session_data:
        save_to_csv_with_period(current_session_data, OUTPUT_FILE)
        save_progress(newly_processed_puuids)

    # 최종 중복 제거
    final_cleanup_by_period(OUTPUT_FILE)

## 샘플 유저 데이터 서버마다 합치기

In [ ]:
import warnings
warnings.filterwarnings('ignore')

def perform_stratified_sampling(sample_fraction, input_file, output_file, min_per_group=1, stratify_col='tier', random_state=42):
    """
    계층적 샘플링 수행 (기본: 티어별 샘플링).
    - sample_fraction: 각 그룹에서 비율로 뽑을 비율
    - min_per_group: 그룹이 작을 때 최소 보장 샘플 수
    - stratify_col: 계층화에 사용할 컬럼 이름
    """
    print('📂 데이터 로딩 중...')
    df = pd.read_csv(input_file)

    if stratify_col not in df.columns:
        raise ValueError(f"'{stratify_col}' 컬럼이 입력 파일에 없습니다.")

    # 그룹별로 샘플 수 계산
    def _sample_group(g):
        grp_size = len(g)
        n = max(0, int(round(grp_size * sample_fraction)))
        if n < min_per_group and grp_size > 0:
            n = min(min_per_group, grp_size)
        if n <= 0:
            return g.iloc[0:0]
        return g.sample(n=n, random_state=random_state)

    sampled_df = df.groupby(stratify_col, group_keys=False).apply(_sample_group).reset_index(drop=True)

    # 결과 출력 및 저장
    print('\n✅ 샘플링 완료!')
    print(f"전체 데이터 수: {len(df):,} 명")
    print(f"샘플링 데이터 수: {len(sampled_df):,} 명")
    print(f"샘플 내 {stratify_col} 비중:\n{(sampled_df[stratify_col].value_counts(normalize=True) * 100).round(2)}%")

    sampled_df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"💾 샘플 데이터가 {output_file}에 저장되었습니다.")

In [ ]:
# 환경에 맞게 파일 경로 입력
file_paths = [
    'data/01_puuids/LA2_puuids_high_tiers',
    'data/01_puuids/LA2_puuids_iron_diamond',
    'data/01_puuids/JP1_puuids_high_tiers',
    'data/01_puuids/JP1_puuids_iron_diamond',
    'data/01_puuids/ME1_puuids_high_tiers',
    'data/01_puuids/ME1_puuids_iron_diamond',
    'data/01_puuids/OC1_puuids_high_tiers',
    'data/01_puuids/OC1_puuids_iron_diamond',
    'data/01_puuids/RU_puuids_high_tiers',
    'data/01_puuids/RU_puuids_iron_diamond',
    'data/01_puuids/SG2_puuids_high_tiers',
    'data/01_puuids/SG2_puuids_iron_diamond',
    'data/01_puuids/TW2_puuids_high_tiers',
    'data/01_puuids/TW2_puuids_iron_diamond',
]

for name in file_paths:
    '''샘플링 비율은 조정 필요함'''
    sampling = 0.005
    perform_stratified_sampling(
        sample_fraction = sampling,
        input_file=f'{name}.csv', 
        output_file=f'{name}_{sampling*100}%.csv'
    )

In [6]:
# 샘플링 된 유저데이터 로드
la2_high=pd.read_csv('data/01_puuids/LA2_puuids_high_tiers_0.5%.csv')
sg2_high=pd.read_csv('data/01_puuids/SG2_puuids_high_tiers_0.5%.csv')
jp1_high=pd.read_csv('data/01_puuids/JP1_puuids_high_tiers_0.5%.csv')
tw2_high=pd.read_csv('data/01_puuids/TW2_puuids_high_tiers_0.5%.csv')
ru_high=pd.read_csv('data/01_puuids/RU_puuids_high_tiers_0.5%.csv')
oc1_high=pd.read_csv('data/01_puuids/OC1_puuids_high_tiers_0.5%.csv')
me1_high=pd.read_csv('data/01_puuids/ME1_puuids_high_tiers_0.5%.csv')

la2_low=pd.read_csv('data/01_puuids/LA2_puuids_iron_diamond_0.5%.csv')
sg2_low=pd.read_csv('data/01_puuids/SG2_puuids_iron_diamond_0.5%.csv')
jp1_low=pd.read_csv('data/01_puuids/JP1_puuids_iron_diamond_0.5%.csv')
tw2_low=pd.read_csv('data/01_puuids/TW2_puuids_iron_diamond_0.5%.csv')
ru_low=pd.read_csv('data/01_puuids/RU_puuids_iron_diamond_0.5%.csv')
oc1_low=pd.read_csv('data/01_puuids/OC1_puuids_iron_diamond_0.5%.csv')
me1_low=pd.read_csv('data/01_puuids/ME1_puuids_iron_diamond_0.5%.csv')

In [7]:
regions = ["LA2", "SG2", "JP1", "TW2", "RU", "OC1", "ME1"]

low_df = [la2_low, sg2_low, jp1_low, tw2_low, ru_low, oc1_low, me1_low]
high_df = [la2_high, sg2_high, jp1_high, tw2_high, ru_high, oc1_high, me1_high]

# 최종적으로 사용할 딕셔너리
df_dict = {}

# zip 활용
for region, low, high in zip(regions, low_df, high_df):
    
    # 컬럼 삭제
    cols_to_drop = ['division', 'leagueId']
    low.drop(columns=cols_to_drop, errors='ignore', inplace=True)
    high.drop(columns=cols_to_drop, errors='ignore', inplace=True)
    
    # Low와 High 합치기
    combined = pd.concat([low, high], ignore_index=True)
    
    # 중복 유저(puuid)가 있을 수 있으니 한 번 더 제거
    combined.drop_duplicates(subset=['puuid'], inplace=True)
    
    # 딕셔너리에 저장 (Key: 서버이름, Value: 통합 데이터프레임)
    df_dict[region] = combined

print(f"✅ {len(df_dict)}개 지역의 데이터 통합 완료!")
print(f"대상 지역: {list(df_dict.keys())}")

✅ 7개 지역의 데이터 통합 완료!
대상 지역: ['LA2', 'SG2', 'JP1', 'TW2', 'RU', 'OC1', 'ME1']


## EXE

In [ ]:
collect_by_patch_periods(df_dict)

## 매치데이터

In [10]:
matchids = pd.read_csv('data/02_match-v5/collected_match_ids.csv')

In [11]:
print(matchids.head())

         match_id    period
0  LA2_1501465065  period_2
1  LA2_1501457348  period_2
2  LA2_1501453622  period_2
3  LA2_1501455030  period_2
4  LA2_1501453109  period_2


In [ ]:
# url = f"https://{routing}.api.riotgames.com/lol/match/v5/matches/{matchid}?api_key={API_KEY}"